In [1]:
import re
import wandb
import pandas as pd

metric = "Objective/eval_all"
step_threshold = 150 # 75 for tlr

name = "eval_vit_in21k_act"
name = name.replace('eval', 'vo')

name = "vo_cnn_in21k_fwd"

In [2]:
# retreive runs

run_ids = []
run_names = []
run_ckpts = []

api = wandb.Api()
runs = api.runs("memmelma/vo")

for run in runs:
    if name in run.name:
        run_ids += [run.id]
        run_names += [run.name]
        run_ckpts += [run.config["config"]["CHECKPOINT_FOLDER"]]
run_ids, run_names

(['202yhn9i'], ['vo_cnn_in21k_fwd[1/1]'])

In [3]:
# filter runs

pattern = name + "\[.\/.]"

run_names_filter = []
run_ids_filter = []
run_ckpts_filter = []

for run_name, run_id, run_ckpt in zip(run_names,run_ids,run_ckpts):
    if re.match(pattern, run_name) is not None:
        run_names_filter += [run_name]
        run_ids_filter += [run_id]
        run_ckpts_filter += [run_ckpt]

In [4]:
run_names_filter

['vo_cnn_in21k_fwd[1/1]']

In [5]:
# combine runs

infos = pd.DataFrame([])

for run_id, run_ckpt in zip(run_ids_filter, run_ckpts_filter):
    
    
    run = api.run(f"/memmelma/vo/runs/{run_id}")
    
    history = run.history()
    select = history["_step"]<=step_threshold
    
    info = pd.concat((history[metric][select], history["_step"][select]), axis=1)
    info.columns = [metric, "_step"]
    
    info["CHECKPOINT_FOLDER"] = run_ckpt

    infos = pd.concat((infos, info))

In [6]:
name

'vo_cnn_in21k_fwd'

In [7]:
# get best run
best_run = infos[infos[metric] == infos[metric].min()]
best_run

,Objective/eval_all,_step,CHECKPOINT_FOLDER
149,0.002847,149,train_log/cnn/vo_cnn_in21k_fwd_dt_20220415_085...


In [8]:
# print checkpoint
print('\n      "regress_all_best": { \n        "all": "' + best_run["CHECKPOINT_FOLDER"].item() + f'/ckpt_epoch_{best_run["_step"].item()}'+'.pth",\n        },')


      "regress_all_best": { 
        "all": train_log/cnn/vo_cnn_in21k_fwd_dt_20220415_085622607515_s_1/checkpoints/ckpt_epoch_149.pth,
        },
